- You'll need Chrome driver for this. Install it in the working directory or any directory and pass the directory  like this  webdriver.Chrome('chromedriver.exe') as I did in the get_the_translation function
- Pip install pytesseract, pip install tesseract
- Install tesseract OCR for your OS and add its path to the environment variable.

In [ ]:
import cv2
import pytesseract
from selenium import webdriver

In [ ]:
dict_={'tamil':'ta','english':'en','spanish':'es','hindi':'hi','french':'fr'}

#### Getting the translation for the input from google translation using selenium

In [ ]:
def get_the_translation(from_,to_,Text):
    global driver
    driver=webdriver.Chrome('chromedriver.exe')
    driver.get('https://translate.google.com/#view=home&op=translate&sl={}&tl={}&text={}'.format(dict_[from_],dict_[to_],Text))
    first=driver.find_elements_by_xpath("""//html/body/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/div[2]""")
    return first[0].text

In [ ]:

def main(frm,to):
    cap=cv2.VideoCapture(0)
    meaning=""

    if cap.isOpened():
        ret,frame=cap.read()
    else:
        ret=False
    while ret:
        ret,frame=cap.read()
        orig=frame.copy()
         # Select ROI
        cv2.putText(frame,"{}-->{}".format(frm,to),(10,30),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)

        fromCenter = False
        if cv2.waitKey(1) & 0xFF == ord("s"):
            #Selecting the word you want to translate from the screen
            r = cv2.selectROI("Selection_frame",frame, fromCenter)

            # Crop image
            imCrop = frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
            gray = cv2.cvtColor(imCrop, cv2.COLOR_BGR2GRAY)
            gray = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
            Text = pytesseract.image_to_string(gray)
            meaning=get_the_translation(frm,to,Text)

            driver.close()
        if meaning is not None:
                cv2.putText(frame,"translation: {}".format(meaning),(20,450),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)

        cv2.imshow('f',frame)
        if cv2.waitKey(1)==27:
            break
    cv2.destroyAllWindows()
    cap.release()
